<a href="https://colab.research.google.com/github/schauppi/Udemy_TensorFlow_Developer/blob/main/Abschnitt_8_NLP_fundamentals_in_TensorFlow/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (text or speech sequences).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-52d1b947-0433-0f8c-a238-1e753c5b38db)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-08-05 14:09:01--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-05 14:09:02 (55.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we´re going to be using is Kaggle´s introduction to BLP dataset (text samples of Tweets labelled as disaster or nor disaster)

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2021-08-05 14:09:04--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.157.128, 74.125.23.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.157.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-08-05 14:09:05 (101 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset 

To visualize our text samples, we first have to read them in, one way to do so would be to use Python.

Another way to do this is to use pandas...

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
#What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let´s visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) #create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
@OllyMursAus I do feel sorry for him! He is not a piece of meat! He is a nice guy... People don't need to rush him and screams in his face!

---

Target: 0 (not real disaster)
Text:
@Mmchale13 *tries to electrocute self with phone cord*

---

Target: 1 (real disaster)
Text:
@SatanaOfHell ever seen by far. A dreamy look came over his face and his ears flattened as he started to walk slowly toward her.* You'd-

---

Target: 1 (real disaster)
Text:
Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/VBsj8tniv1

---

Target: 0 (not real disaster)
Text:
I'm not a Drake fan but I enjoy seeing him body-bagging people. Great marketing lol.

---



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split

#Use train_test_split to split train data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [13]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
#Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Convert text into numbers

When dealing with a text problem, one of the first things you´ll have to do before you can build a model is to convert your text to numbers.

There are a few ways you can to this, namely:
* Tokenization - direct mapping of token (a token could be a word or character to number)
* Embedding - create a matrix of a feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

## Text vectorization (tokenization)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, #how man word in the vocabulary (automatically adds <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, #create groups of n-words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, #how long the sequences should be
                                    pad_to_max_tokens=True)

In [16]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [17]:
len(train_sentences[0].split())

7

In [18]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
#Setup vectorizeation variables
max_vocab_length = 10000 #max number of words to have in our vocabulary
max_length = 15 #max length our sequences will be (e.g. how many words from a tweet will our model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [21]:
#Create a sample sentence and tokenize it
sample_sentence = "There´s a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 @gideonstrumpet Have you been held hostage?       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,   24,   12,   59, 2095,  509,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [23]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] #get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words
print(len(words_in_vocab)) 
print(top_5_words)
print(bottom_5_words)

10000
['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we´re going to use TensorFlow´s embedding layer.

The parameters we care most about for our embedding layer:
* input_dim = the size of the vocabulary
* output_dim = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [24]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

In [25]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence} \
      \n\nEmbedded version")
#Embed the random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 God bless you and your mudslide cake Dorret ????       

Embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04350764,  0.0041792 ,  0.00143475, ..., -0.01190903,
          0.0132598 , -0.04723803],
        [-0.0101433 ,  0.00393796, -0.00554855, ..., -0.04908801,
          0.03784659, -0.02898551],
        [-0.0252121 , -0.01277481, -0.03647538, ..., -0.03140043,
         -0.00172595, -0.01226259],
        ...,
        [-0.04571716, -0.02992847, -0.00683478, ..., -0.01905673,
          0.00538705, -0.0292891 ],
        [-0.04571716, -0.02992847, -0.00683478, ..., -0.01905673,
          0.00538705, -0.0292891 ],
        [-0.04571716, -0.02992847, -0.00683478, ..., -0.01905673,
          0.00538705, -0.0292891 ]]], dtype=float32)>

In [26]:
# Check out a single token´s embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04350764,  0.0041792 ,  0.00143475, -0.0330928 ,  0.00937252,
         0.03700695,  0.00044922, -0.0004993 ,  0.03794819, -0.04394536,
        -0.02855834, -0.03508487,  0.00252682, -0.04443464, -0.0170136 ,
         0.0429129 ,  0.03239917, -0.01630409, -0.02272826,  0.02529353,
         0.03651224,  0.0369066 , -0.01081877,  0.00781699,  0.03934968,
        -0.03378121, -0.02736526, -0.02932651,  0.04868977,  0.04604704,
        -0.03606639, -0.01325725,  0.04283677, -0.03479545,  0.0086277 ,
        -0.03031603, -0.03789209, -0.04833254,  0.00617057, -0.04200392,
        -0.0213644 , -0.02585703,  0.00834946,  0.02576092,  0.04896336,
         0.01324039,  0.00739849, -0.00219845,  0.02462789, -0.01567277,
         0.02240512,  0.00147091, -0.01779213,  0.00096519, -0.02662557,
        -0.03820987, -0.00148237,  0.03702463,  0.0160036 ,  0.00453075,
         0.04927999,  0.00888499, -0.02518618,  0.04486069,  0.04990909,
  

## Modelling a text dataset (running a series of experiments)

Now we´ve got a way to turn our text sequences into numbers, it´s time to start building a series of modelling experiments.

We´ll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM
* Model 3: GRU
* Model 4: Bidirectional LSTM
* Model 5: 1D Convolution NN
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data

How are we goint to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit the model
* Evaluate the model

### Model 0: Getting a baseline

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Create tokenization and modellin pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), #cobvert words to numbers
                    ("clf", MultinomialNB()) #model the text
])

#Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [28]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [29]:
#Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

In [30]:
#Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 scroe of a binary classification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, racall and f1-scroe using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}

  return model_results

In [31]:
#Get baseline results
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [32]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [33]:
# Build model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) #inputs are 1-dimensional strings
x = text_vectorizer(inputs) #turn the input text into numbers
x = embedding(x) #create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) #create the feature vector for each token
outputs = layers.Dense(1, activation="sigmoid")(x) #Create the binary output
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile model
model_1.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [36]:
#Fit the model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210805-140914
Epoch 1/5
215/215 [==============================] - 8s 22ms/step - loss: 0.6101 - accuracy: 0.6892 - val_loss: 0.5353 - val_accuracy: 0.7493
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.4402 - accuracy: 0.8205 - val_loss: 0.4689 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.3455 - accuracy: 0.8610 - val_loss: 0.4645 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2837 - accuracy: 0.8917 - val_loss: 0.4624 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2362 - accuracy: 0.9133 - val_loss: 0.4829 - val_accuracy: 0.7861


In [37]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4829 - accuracy: 0.7861


[0.4828896224498749, 0.7860892415046692]

In [38]:
#Make some predictions and evaluate those
model_1_preds_probs = model_1.predict(val_sentences)
model_1_preds_probs.shape

(762, 1)

In [39]:
#look at a single prediction
model_1_preds_probs[0]

array([0.25375122], dtype=float32)

In [40]:
#look at the first 10 predictions
model_1_preds_probs[:10]

array([[0.25375122],
       [0.7912575 ],
       [0.9977896 ],
       [0.13534513],
       [0.09633459],
       [0.94518566],
       [0.92732334],
       [0.9930212 ],
       [0.9596207 ],
       [0.33421803]], dtype=float32)

In [41]:
#Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_preds_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [42]:
# Calculate our model_1 results
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'f1': 0.7827032509906072,
 'precision': 0.7920916944370863,
 'recall': 0.7860892388451444}

In [43]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [44]:
# Get the vocabulary from the text vectorization
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [46]:
#Get weight matrix of embedding layer
#these are the numerical representations of each token in our training data
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) #same size as vocav size and embedding dim

(10000, 128)


To visualize, TensorFlow has a handy tool calles projector : https://projector.tensorflow.org/

In [47]:
#Create embedding files

import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [48]:
#Download files from colab to upload to projector
"""
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
"""

"\ntry:\n  from google.colab import files\n  files.download('vectors.tsv')\n  files.download('metadata.tsv')\nexcept Exception:\n  pass\n"

## Recurrent Neural Networks (RNN´s)

The premise of a RNN is to use the representation of a previous input to aid the representation of the later input.

### Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typicall looks like this:

Input (text) - Tokenize - Embedding - Layers (RNN/dense) - Output (label probability)

In [60]:
#Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.LSTM(64, return_sequences=True)(x) #when you´re stacking RNN cells together, you need to set return_sequences=True
x = layers.LSTM(64, return_sequences=False)(x)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [61]:
#Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [62]:
#Compile the model
model_2.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [63]:
#Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210805-143323
Epoch 1/5
215/215 [==============================] - 10s 27ms/step - loss: 0.2202 - accuracy: 0.9240 - val_loss: 0.5900 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.1542 - accuracy: 0.9423 - val_loss: 0.5814 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 5s 22ms/step - loss: 0.1268 - accuracy: 0.9524 - val_loss: 0.6270 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 5s 21ms/step - loss: 0.1056 - accuracy: 0.9596 - val_loss: 0.8665 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0846 - accuracy: 0.9663 - val_loss: 1.0244 - val_accuracy: 0.7703


In [64]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[5.0793430e-03],
       [7.7724391e-01],
       [9.9944347e-01],
       [2.4860829e-02],
       [3.4457006e-04],
       [9.9769127e-01],
       [9.0563548e-01],
       [9.9966431e-01],
       [9.9943930e-01],
       [4.5516431e-01]], dtype=float32)

In [65]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [68]:
#Calculate model 2 results
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 77.03412073490814,
 'f1': 0.7671895343790688,
 'precision': 0.7743290047665735,
 'recall': 0.7703412073490814}

In [69]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [76]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=False)(x)
#x = layers.LSTM(64)(x)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [77]:
#get model summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru_4 (GRU)                  (None, 64)                37248     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [78]:
#Compile the model
model_3.compile(loss="binary_crossentropy", optimizer="Adam", metrics="accuracy")

In [79]:
#Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20210805-151752
Epoch 1/5
215/215 [==============================] - 8s 28ms/step - loss: 0.1511 - accuracy: 0.9421 - val_loss: 0.7810 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0825 - accuracy: 0.9692 - val_loss: 0.8838 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0699 - accuracy: 0.9739 - val_loss: 0.8624 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0593 - accuracy: 0.9768 - val_loss: 0.8982 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0497 - accuracy: 0.9793 - val_loss: 1.3396 - val_accuracy: 0.7730


In [81]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[7.2019192e-04],
       [5.7174861e-01],
       [9.9991918e-01],
       [2.1735054e-02],
       [8.3342587e-05],
       [9.9985409e-01],
       [9.6660531e-01],
       [9.9995100e-01],
       [9.9993134e-01],
       [9.8258120e-01]], dtype=float32)

In [82]:
#Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [83]:
#Calculate model 3 results
model_3_results = calculate_results(val_labels, model_3_preds)
model_3_results

{'accuracy': 77.29658792650919,
 'f1': 0.7704381702262061,
 'precision': 0.775649278370019,
 'recall': 0.7729658792650919}